In [45]:
import pandas as pd
from random import randint
from random import uniform
from sympy.solvers import solve
from sympy import Symbol

In [46]:
class Bird:
    def __init__(self, species):
        self.__score = 0
        self.__games = 0
        
        self.setSpecies(species)
        self.setStrat()

    def getSpecies(self):
        return self.__species
    
    def getStrat(self):
        return self.__strat
    
    def getScore(self):
        return self.__score
    
    def getGames(self):
        return self.__games

    def setSpecies(self, species):
        self.__species = species
        
    def setStrat(self):
        if self.getSpecies() == 'Dove':
            self.__strat = 0
            
        elif self.getSpecies() == 'Hawk':
            self.__strat = 1
            
        else:
            self.__strat = randint(0, 1)
        
    def incScore(self, points):
        self.__score += points
        
    def incGames(self):
        self.__games += 1

In [47]:
def Fight(player, opp):
    if player.getSpecies() != 'Hawk' and player.getSpecies() != 'Dove':
        player.setStrat()
        
        if opp.getSpecies() != 'Hawk' and opp.getSpecies() != 'Dove':
            if player.getStrat() == 0:
                opp.setSpecies('Hawk')
                
            else:
                opp.setSpecies('Dove')
                
            opp.setStrat()
    
    if player.getStrat() == 0 and opp.getStrat() == 0:
        player.incScore(-10)
        chance = randint(0, 1)

        if chance == 1:
            player.incScore(50)
            
    elif player.getStrat() == 1:
        if opp.getStrat() == 0:
            player.incScore(50)
        
        else:
            battle = randint(0, 1)

            if battle == 1:
                player.incScore(50)
            
            else:
                player.incScore(-100)
            
    player.incGames()

## Expectation Table

In [48]:
birdSpecies = ['Hawk', 'Dove', 'Bourgeois']
warbirds = {}

for flyer in birdSpecies:
    opponents = {}
    
    for opp in birdSpecies:
        player = Bird(flyer)
        
        for i in range(100000):
            enemy = Bird(opp)
            Fight(player, enemy)
        
        avg = player.getScore() / player.getGames()
        opponents[opp] = avg

    warbirds[flyer] = opponents
    
warbirds

{'Hawk': {'Hawk': -24.883, 'Dove': 50.0, 'Bourgeois': 12.14},
 'Dove': {'Hawk': 0.0, 'Dove': 15.104, 'Bourgeois': 7.3261},
 'Bourgeois': {'Hawk': -12.4295, 'Dove': 32.5319, 'Bourgeois': 25.156}}

In [49]:
scoreTable = pd.DataFrame(columns = birdSpecies)

for bird in warbirds:
    scoreTable.loc[bird] = warbirds[bird].values()

scoreTable

,Hawk,Dove,Bourgeois
Hawk,-24.8830,50.0000,12.1400
Dove,0.0000,15.1040,7.3261
Bourgeois,-12.4295,32.5319,25.1560


## Stable Population

In [50]:
hawkScore = int(round(scoreTable.Hawk[0]))
doveScore = int(scoreTable.Dove[0])

P = Symbol('P')
expr = hawkScore * P + doveScore * (1 - P)

sol = solve(expr)
sol[0]

2/3

In [51]:
birdSpecies = ['Hawk', 'Dove']
warbirds = {}

for flyer in birdSpecies:
    player = Bird(flyer)
    
    for i in range(100000):
        prob = float(sol[0])
        spawn = uniform(0, 1)
        
        if spawn <= prob:
            enemy = Bird(birdSpecies[0])
            
        else:
            enemy = Bird(birdSpecies[1])
            
        Fight(player, enemy)

    avg = player.getScore() / player.getGames()
    warbirds[flyer] = avg
    
warbirds

{'Hawk': 0.0365, 'Dove': 4.9977}